In [1]:
import torch
import pickle
from pprint import pprint

# 1. Datasets

In [2]:
# Source seq
with open('../datasets/test/en', 'r+') as file_obj:
    data_en = file_obj.readlines()
data_en = [ line.strip().lower() for line in data_en ]

print(len(data_en))
print(data_en[:5])

101636
['ive been thinking about buying new towels.', 'ive been thinking about buying new towels.', 'base jumping is an extreme sport.', 'dancing is basically a rigorous sport.', "i don't know and i don't care."]


In [3]:
# Target seq
with open('../datasets/test/vi', 'r+') as file_obj:
    data_vi = file_obj.readlines()
data_vi = [ line.strip().lower() for line in data_vi ]

print(len(data_vi))
print(data_vi[:5])

101636
['tôi đang nghĩ về việc mua khăn mới.', 'tôi đã suy nghĩ về việc mua khăn mới.', 'nhảy cầu là một môn thể thao mạo hiểm.', 'nhảy cơ bản là một môn thể thao khắc nghiệt.', 'tôi không biết và tôi không quan tâm.']


# 2. Translate single sentence

In [5]:
## Load model
transformer = torch.load('ckpts/8B_Transformer_en2vi.model.pth')
transformer = transformer.to('cpu')
transformer.eval()

## Load tokenizers
from transformers import AutoTokenizer

en_tokenizer = AutoTokenizer.from_pretrained(
    'bert-base-uncased')
vi_tokenizer = AutoTokenizer.from_pretrained(
    'vinai/phobert-base')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
